In [1]:
%load_ext autoreload
%autoreload 2

import ast
from pathlib import Path
from pprint import pprint

In [3]:
from docstring_format import *
from docstring_format.base import *
from docstring_format.constants import *
import json

In [4]:
def get_functions(raw_text: str):
    # TODO walk the nodes ?
    tree = ast.parse(raw_text)
    
    functions = [item for item in tree.body if isinstance(item, ast.FunctionDef)]
    classes = [item for item in tree.body if isinstance(item, ast.ClassDef)]
    class_methods = [func for item in classes for func in item.body if isinstance(func, ast.FunctionDef)]
    functions.extend(class_methods)
    return functions

In [5]:
file = Path('./tests/dummy_tests_functions.py')
raw_text = file.read_text()
functions = get_functions(raw_text)
dirty_lines = raw_text.splitlines()

In [6]:
docstrings = []
for func in functions:    
    docstrings.append(Docstring.from_ast(func, dirty_lines))

In [8]:
for docstring in docstrings:
    docstring.annotate()

In [9]:
pprint(docstrings)

[Docstring(function=<ast.FunctionDef object at 0x000002879D5EF4C0>,
           lines=['    """',
                  '    AZrojrltndflg lejkkjntgdf',
                  '',
                  '    Parameters',
                  '    ----------',
                  '    arg1: test',
                  '',
                  '    Returns',
                  '    -------',
                  '',
                  '    """'],
           start=4,
           length=11,
           offset='    ',
           sections=[Section(name='Start',
                             type=<SectionType.SUMMARY: 2>,
                             start=0,
                             length=3,
                             offset='    ',
                             lines=['    """',
                                    '    AZrojrltndflg lejkkjntgdf',
                                    ''],
                             annotation=None,
                             corrected_lines=['    AZrojrltndflg lejkkjntgdf']),
      

In [ ]:
docstring = docstrings[0]
pprint(docstring.sections)

In [ ]:
[line for item in docstring.sections for line in item.lines]

- Look for type annotation
- if is one-liner, transform to two line comment

In [ ]:

def clean_lines_section(lines: list[str]):
    # remove docstring tag at the first and last line
    lines = lines.copy()
    first_line = lines[0]
    match = re.search('\s*["\']{3}(.*)', first_line)
    if match:
        lines[0] = match.groups()[0]

    last_line = lines[-1]
    match = re.search('\s*(.*)["\']{3}', last_line)
    if match:
        lines[-1] = match.groups()[0]

    # remove trailing empty lines
    for _ in range(len(lines)):
        match = re.search('^\s*$', lines[0])
        if match:
            lines.pop(0)
        else:
            break

    for _ in range(len(lines)):
        match = re.search('^\s*$', lines[-1])
        if match:
            lines.pop(-1)
        else:
            break
    return lines


def annotate_section(section: 'Section'):
    section.corrected_lines = clean_lines_section(section.lines)
    if section.type in ANNOTATE_MAP.keys():
        return ANNOTATE_MAP[section.type](section)
    return section


def annotate_arg_section(section: 'Section'):
    lines = section.corrected_lines.copy() if section.corrected_lines is not None else section.lines.copy()
    pattern = re.compile(f'{section.name}\s*:\s*{section.annotation}')
    offset = section.offset
    # no annotation discovered
    if not any([re.search(pattern, line) for line in lines]):
        # Are description and variable on the same line
        line = lines.pop(0)
        match = re.search(f'\s*{section.name}\s*:\s*(.*)', line)
        if match:
            description = match.groups()[0]
            lines = [offset + f'{section.name} : {section.annotation}',
                     offset * 2 + description.capitalize()] + lines
        else:
            lines = [offset + f'{section.name} : {section.annotation}'] + lines
    section.corrected_lines = lines
    return section

In [ ]:
def annotate_returns(section: 'Section'):
    lines = section.corrected_lines.copy() if section.corrected_lines is not None else section.lines.copy()
    offset = section.offset
    
    # remove first line delimiters
    for _ in range(len(lines)):
        line = lines[0]
        if (re.search('[Rr]eturns?', line) or 
            re.search('-+', line)):
            lines.pop(0)

    # remove whitespaces and add twice the offset
    lines = [offset*2 + re.search('\s(.*)', line).group() for line in lines]
    if section.annotation:
        pattern = re.compile(f'{section.annotation}')
        if not any([re.search(pattern, line) for line in lines]):
            lines.insert(0, offset + section.annotation)
        
    # add return delimiter
    lines = [offset+'Returns', offset+'-------'] + lines

    section.corrected_lines = lines
    return section

In [ ]:
ANNOTATE_MAP = {SectionType.ARG: annotate_arg_section,
                SectionType.RETURNS: annotate_returns}

In [ ]:
for section in docstring.sections:
    annotate_section(section)

In [ ]:
pprint(docstring.sections[-1])

In [ ]:
annotate_returns(docstring.sections[-1])